# PROJETO 5 - ANÁLISE DE LESÕES NA PELE COM INTELIGÊNCIA ARTIFICIAL

Neste projeto será feito uma classificação multiclasses de diferentes tipos de lesões na pele humana. Para isso, será usando modelos de deep learning com arquitetura CNN e Densenet. 

Fonte dos dados: https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000

## 1. Instalando e carregando pacotes

In [1]:
# Versão da linguagem Python
from platform import python_version
print('A versão da linguagem Python utilizada neste Jupyter Notebook é: ', python_version())

A versão da linguagem Python utilizada neste Jupyter Notebook é:  3.9.13


In [2]:
# Instala Pytorch
!pip install -q torch==1.13.0

In [3]:
# Instala Torchvision
!pip install -q torchvision==0.14.0

In [4]:
# Instala Lightning
!pip install -q pytorch-lightning==1.8.3

In [6]:
# Importa os pacotes/funções

# Manipulação das imagens
import os # Manipula o Sistema Operacional
import cv2 # Converte imagens em dados
import itertools # Iteração dos dados
from tqdm import tqdm # Barra de progressão
from glob import glob # Manipulação de imagens
from PIL import Image # Manipulação de imagens

# Manipulação e visualização dos dados
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Pytorch
import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Pacotes para o relatório de hardware
import gc
import types
import pkg_resources
import pytorch_lightning as pl

# Seend para reprodução dos resultados DSA
np.random.seed(10)
torch.manual_seed(10)
torch.cuda.manual_seed(10)

In [7]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Projeto 5 - IA para Análise de Imagens na Pele" --iversions

Author: Projeto 5 - IA para Análise de Imagens na Pele

torch            : 1.13.0
cv2              : 4.7.0
matplotlib       : 3.5.2
numpy            : 1.21.5
torchvision      : 0.14.0
pytorch_lightning: 1.8.3
PIL              : 9.2.0
pandas           : 1.4.4



## 2. Verificando o Hardware

In [8]:
# Relatório completo

# Verificando o dispositivo
processing_device = "cuda" if torch.cuda.is_available() else "cpu"

# Verificando se GPU pode ser usada (isso depende da plataforma CUDA estar instalada)
torch_aval = torch.cuda.is_available()

# Labels para o relatório de verificação
lable_1 = 'Visão Geral do Ambiente'
lable_2 = 'Se NVIDIA-SMI não for encontrado, então CUDA não está disponível'
lable_3 = 'Fim da Checagem'

# Função para verificar o que está importado nesta sessão
def get_imports():

    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):            
            name = val.__module__.split(".")[0]

        poorly_named_packages = {"PIL": "Pillow", "sklearn": "scikit-learn"}

        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name

# Imports nesta sessão
imports = list(set(get_imports()))

# Loop para verificar os requerimentos
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))
        
# Pasta com os dados (quando necessário)
pasta_dados = r'dados'

print(f'{lable_1:-^100}')
print()
print(f"Device:", processing_device)
print(f"Pasta de Dados: ", pasta_dados)
print(f"Versões dos Pacotes Requeridos: ", requirements)
print(f"Dispositivo Que Será Usado Para Treinar o Modelo: ", processing_device)
print(f"CUDA Está Disponível? ", torch_aval)
print("Versão do PyTorch: ", torch.__version__)
print("Versão do Lightning: ", pl.__version__)
print()
print(f'{lable_2:-^100}\n')
!nvidia-smi
gc.collect()
print()
print(f"Limpando a Memória da GPU (se disponível): ", torch.cuda.empty_cache())
print("\nModelo da GPU:")
# Modelo da GPU usada
!nvidia-smi --query-gpu=name --format=csv,noheader
print(f'\n{lable_3:-^100}')

--------------------------------------Visão Geral do Ambiente---------------------------------------

Device: cpu
Pasta de Dados:  dados
Versões dos Pacotes Requeridos:  [('Pillow', '9.2.0'), ('tqdm', '4.64.1'), ('matplotlib', '3.5.2'), ('numpy', '1.21.5'), ('torch', '1.13.0'), ('pandas', '1.4.4'), ('torchvision', '0.14.0')]
Dispositivo Que Será Usado Para Treinar o Modelo:  cpu
CUDA Está Disponível?  False
Versão do PyTorch:  1.13.0+cpu
Versão do Lightning:  1.8.3

------------------Se NVIDIA-SMI não for encontrado, então CUDA não está disponível------------------


Limpando a Memória da GPU (se disponível):  None

Modelo da GPU:


'nvidia-smi' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.



------------------------------------------Fim da Checagem-------------------------------------------


'nvidia-smi' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


## 3. Interpretação dos metadados

In [12]:
# Leitura do arquivo de metadados
df_inicial = pd.read_csv(os.path.join('dados', 'HAM10000_metadata.csv'))

# Visualiza o registro
df_inicial.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear
